##### Author: Sailesh Chauhan
##### Date  : 01/08/2021
##### Title : Twitter live tweet streaming to Azure Event Hub and sentiment analysis using TextBlob library

##### Importing required Libraries

In [ ]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import nltk
from textblob import TextBlob, Word, Blobber
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

##### Running Custom Logging for logging info

In [ ]:
%run ./custom_logging

##### Method for analyzing text received in twitter stream.

In [ ]:
def getblobsentiment(text):
  '''
  Description:
    Method completes sentiment analysis on text in parameters.
  Paramater:
    Text for sentiment analysis.
  Returns:
    if polarity is greater than 0 return 1
    if polarity is lesser than 0 return 1
    if polarity is 0 than return 0
  '''
  try:
    blob = TextBlob(text)
    i=0.00
    for sentence in blob.sentences:
        i=i+sentence.sentiment.polarity

    if i >0: 
      return 1
    elif i<0:
      return -1
    else:
      return 0
  except Exception as ex:
      logger.DEBUG('Failed to get senitment '+str(ex))

  
async def run(text):
  '''
  Description:
    Method runs asyncronously using asyncio,EventHubProducerClient for creating connection with eventhub.
  Parameter:
    text from twitter tweet.
  Return:
    None.
  '''
  try:
    producer = EventHubProducerClient.from_connection_string(conn_str=secretKeys['conn_str'], eventhub_name=secretKeys['eventhub_name'])
    async with producer:
      event_data_batch = await producer.create_batch()
      event_data_batch.add(EventData(text))
      await producer.send_batch(event_data_batch)
  except Exception as ex:
    logger.debug(' Failed to get events and transmit to EventHub '+str(ex))


##### Running other notebook required for getting secretKeys variable

In [ ]:
%run ./environment

##### Class listener inhereting from StreamListener class. It has three methods on_data,on_error.

In [ ]:
class listener(StreamListener):
  '''
  class listener Inheriting from StreamListener class of tweepy module.
  '''
  def on_data(self, data):
    '''
    Description:
      Method for sending twitter tweets to client. Method establishes connection with twitter application using required keys.
    Parameter:
      data fetched from twitter.
    Return:
      True.
    '''
    try:
      loop = asyncio.get_event_loop()
      t=json.loads(data)

      tweet = {"text":t["text"] ,"sentiment": getblobsentiment(t["text"]) ,"created_at" :t["created_at"]}
      logging.debug(type(data))     
      loop.run_until_complete(run(json.dumps(d1)))
      logging.debug(tweet)
      return(True)
    except Exception as ex:
      logger.error(str(ex))
   
  def on_error(self, status):
    logging.error(status)
    
consumer_key=secretKeys['consumer_key']
consumer_secret=secretKeys['consumer_secret']
acess_token=secretKeys['acess_token']
acess_token_secret=secretKeys['acess_token_secret']

auth = OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(acess_token, acess_token_secret)

twitterStream = Stream(auth, listener()) 
twitterStream.filter(track=["vaccine"])